# Tipos de datos en Python

La ciencia y la computación basadas en datos requieren comprender cómo se almacenan y manipulan los datos.
Este notebook describe y contrasta cómo se manejan las matrices de datos en el propio lenguaje Python, y cómo NumPy mejora esto.

Los usuarios de Python a menudo se sienten atraídos por su facilidad de uso, una de cuyas piezas es el tipado dinámico.
Mientras que un lenguaje de tipado estático como C o Java requiere que cada variable sea declarada explícitamente, un lenguaje de tipado dinámico como Python se salta esta especificación. Por ejemplo, en C podrías especificar una operación concreta de la siguiente manera:

```C
/* C code */
int result = 0;
for(int i=0; i<100; i++){
    result += i;
}
```

Mientras que en Python la operación equivalente podría escribirse así:

```python
# Python code
result = 0
for i in range(100):
    result += i
```

Fíjate en la principal diferencia: en C, los tipos de datos de cada variable se declaran explícitamente, mientras que en Python los tipos se deducen dinámicamente. Esto significa, por ejemplo, que podemos asignar cualquier tipo de dato a cualquier variable:

```python
# Python code
x = 4
x = "four"
```

Aquí hemos cambiado el contenido de ``x`` de un int a un str. Lo mismo en C provocaría (dependiendo de la configuración del compilador) un error de compilación u otras consecuencias no deseadas:

```C
/* C code */
int x = 4;
x = "four";  // FAILS
```

Este tipo de flexibilidad es una pieza que hace que Python y otros lenguajes de tipado dinámico sean convenientes y fáciles de usar.
Entender *cómo* funciona esto es una parte importante del aprendizaje para analizar datos de forma eficiente y efectiva con Python.
Pero lo que esta flexibilidad de tipos también señala es el hecho de que las variables Python son más que su valor; también contienen información extra sobre el tipo del valor.

## Un número entero en Python es algo más que un número entero

La implementación estándar de Python está escrita en C.
Esto significa que cada objeto Python es simplemente una estructura C inteligentemente disfrazada, que contiene no sólo su valor, sino también otra información. Por ejemplo, cuando definimos un entero en Python, como ``x = 10000``, ``x`` no es sólo un entero "en bruto". En realidad es un puntero a una estructura C compuesta, que contiene varios valores.
Mirando a través del código fuente de Python 3, encontramos que la definición del tipo entero (largo) se parece efectivamente a esto (una vez que las macros C se expanden):

```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```

Un único entero en Python 3 contiene en realidad cuatro piezas:

- ``ob_refcnt``, un recuento de referencias que ayuda a Python a gestionar silenciosamente la asignación y liberación de memoria
- ``ob_type``, que codifica el tipo de la variable
- ``ob_size``, que especifica el tamaño de los siguientes miembros de datos
- ``ob_digit``, que contiene el valor entero real que esperamos que represente la variable Python.

Esto significa que hay una cierta sobrecarga en el almacenamiento de un número entero en Python en comparación con un número entero en un lenguaje compilado como C, como se ilustra en la siguiente figura:

![Integer Memory Layout](img/cint_vs_pyint.png)

Aquí ``PyObject_HEAD`` es la parte de la estructura que contiene el recuento de referencias, el código de tipo y otras piezas mencionadas anteriormente.

Observa la diferencia aquí: un entero en C es esencialmente una etiqueta para una posición en memoria cuyos bytes codifican un valor entero.
Un entero Python es un puntero a una posición en memoria que contiene toda la información del objeto Python, incluyendo los bytes que contienen el valor entero.
Esta información extra en la estructura de enteros de Python es lo que permite codificar Python de forma tan libre y dinámica.
Sin embargo, toda esta información adicional en los tipos Python tiene un coste, que se hace especialmente evidente en las estructuras que combinan muchos de estos objetos.

## Una lista Python es más que una lista

Consideremos ahora lo que ocurre cuando utilizamos una estructura de datos Python que contiene muchos objetos Python.
El contenedor multi-elemento mutable estándar en Python es la lista.
Podemos crear una lista de enteros como sigue:

In [ ]:
L = list(range(10))
L

In [ ]:
type(L[0])

O, lo que es lo mismo, una lista de cadenas:

In [ ]:
L2 = [str(c) for c in L]
L2

In [ ]:
type(L2[0])

Gracias a la tipificación dinámica de Python, podemos incluso crear listas heterogéneas:

In [ ]:
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]

Pero esta flexibilidad tiene un coste: para permitir estos tipos flexibles, cada elemento de la lista debe contener su propia información de tipo, número de referencias y otra información, es decir, cada elemento es un objeto Python completo.
En el caso especial de que todas las variables sean del mismo tipo, mucha de esta información es redundante: puede ser mucho más eficiente almacenar los datos en un array de tipo fijo.
La diferencia entre una lista de tipo dinámico y una matriz de tipo fijo (estilo NumPy) se ilustra en la siguiente figura:

![Array Memory Layout](img/array_vs_list.png)

A nivel de implementación, el array contiene esencialmente un único puntero a un bloque contiguo de datos.
La lista de Python, por otro lado, contiene un puntero a un bloque de punteros, cada uno de los cuales a su vez apunta a un objeto completo de Python como el entero de Python que vimos antes.
De nuevo, la ventaja de la lista es la flexibilidad: dado que cada elemento de la lista es una estructura completa que contiene tanto datos como información de tipo, la lista puede llenarse con datos de cualquier tipo que se desee.
Las matrices de tipo fijo al estilo NumPy carecen de esta flexibilidad, pero son mucho más eficientes para almacenar y manipular datos.

## Matrices de tipo fijo en Python

Python ofrece varias opciones diferentes para almacenar datos en buffers de datos eficientes y de tipo fijo.
El módulo incorporado ``array`` (disponible desde Python 3.3) se puede utilizar para crear matrices densas de un tipo uniforme:

In [ ]:
import array
L = list(range(10))
A = array.array('i', L)
A

Aquí ``'i'`` es un código de tipo que indica que el contenido son números enteros.

Mucho más útil, sin embargo, es el objeto ``ndarray`` del paquete NumPy.
Mientras que el objeto ``array`` de Python proporciona un almacenamiento eficiente de datos basados en arrays, NumPy añade a esto *operaciones* eficientes sobre esos datos.

Empezaremos con la importación estándar de NumPy, bajo el alias ``np``:

In [ ]:
import numpy as np

## Creación de matrices a partir de listas en Python

En primer lugar, podemos utilizar ``np.array`` para crear matrices a partir de listas de Python:

In [ ]:
# matriz de enteros:
np.array([1, 4, 2, 5, 3])

Recuerda que a diferencia de las listas de Python, NumPy está limitado a matrices que contienen el mismo tipo.
Si los tipos no coinciden, NumPy hará una conversión ascendente si es posible (aquí, los enteros se convierten a coma flotante):

In [ ]:
np.array([3.14, 4, 2, 3])

Si queremos establecer explícitamente el tipo de datos del array resultante, podemos utilizar la palabra clave ``dtype``:

In [ ]:
np.array([1, 2, 3, 4], dtype='float32')

Por último, a diferencia de las listas de Python, las matrices de NumPy pueden ser explícitamente multidimensionales; he aquí una forma de inicializar una matriz multidimensional utilizando una lista de listas:

In [ ]:
# las listas anidadas dan lugar a matrices multidimensionales
np.array([range(i, i + 3) for i in [2, 4, 6]])

Las listas interiores se tratan como filas de la matriz bidimensional resultante.

## Crear matrices desde cero

Especialmente para matrices más grandes, es más eficiente crear matrices desde cero usando rutinas incorporadas en NumPy.
Aquí hay varios ejemplos:

In [ ]:
# Crear una matriz de enteros de longitud 10 llena de ceros
np.zeros(10, dtype=int)

In [ ]:
# Crear una matriz de coma flotante 3x5 llena de unos
np.ones((3, 5), dtype=float)

In [ ]:
# Crear una matriz 3x5 llena con 3.14
np.full((3, 5), 3.14)

In [ ]:
# Crear un array lleno con una secuencia lineal
# Empezando en 0, terminando en 20, pasando por 2
# (esto es similar a la función incorporada range())
np.arange(0, 20, 2)

In [ ]:
# Crear una matriz de cinco valores espaciados uniformemente entre 0 y 1
np.linspace(0, 1, 5)

In [ ]:
# Crear una matriz de 3x3 de distribuidos uniformemente
# valores aleatorios entre 0 y 1
np.random.random((3, 3))

In [ ]:
# Crear una matriz 3x3 de valores aleatorios distribuidos normalmente
# con media 0 y desviación estándar 1
np.random.normal(0, 1, (3, 3))

In [ ]:
# Crea una matriz 3x3 de enteros aleatorios en el intervalo [0, 10)
np.random.randint(0, 10, (3, 3))

In [ ]:
# Crear una matriz de identidad 3x3
np.eye(3)

In [ ]:
# Crear un array no inicializado de tres enteros
# Los valores serán los que ya existan en esa posición de memoria
np.empty(3)

## Tipos de datos estándar de NumPy

Los arrays de NumPy contienen valores de un único tipo, por lo que es importante tener un conocimiento detallado de esos tipos y sus limitaciones.
Dado que NumPy está construido en C, los tipos serán familiares para los usuarios de C, Fortran y otros lenguajes relacionados.

Los tipos de datos estándar de NumPy se listan en la siguiente tabla.
Tenga en cuenta que al construir un array, se pueden especificar utilizando una cadena:

```python
np.zeros(10, dtype='int16')
```

O utilizando el objeto NumPy asociado:

```python
np.zeros(10, dtype=np.int16)
```

| Data type	    | Descripción |
|---------------|-------------|
| ``bool_``     | Boolean (True or False) almacenado como byte |
| ``int_``      | Por defecto integer type (same as C ``long``; normalmente ``int64`` or ``int32``)| 
| ``intc``      | Idéntico a C ``int`` (normalmente ``int32`` or ``int64``)| 
| ``intp``      | Integer utilizado para la indexación (igual que C ``ssize_t``; normalmente ``int32`` or ``int64``)| 
| ``int8``      | Byte (-128 to 127)| 
| ``int16``     | Integer (-32768 to 32767)|
| ``int32``     | Integer (-2147483648 to 2147483647)|
| ``int64``     | Integer (-9223372036854775808 to 9223372036854775807)| 
| ``uint8``     | Unsigned integer (0 to 255)| 
| ``uint16``    | Unsigned integer (0 to 65535)| 
| ``uint32``    | Unsigned integer (0 to 4294967295)| 
| ``uint64``    | Unsigned integer (0 to 18446744073709551615)| 
| ``float_``    | Abreviatura de ``float64``.| 
| ``float16``   | Flotante de media precisión: bit de signo, exponente de 5 bits, mantisa de 10 bits| 
| ``float32``   | Flotante de precisión simple: bit de signo, exponente de 8 bits, mantisa de 23 bits| 
| ``float64``   | Flotante de doble precisión: bit de signo, exponente de 11 bits, mantisa de 52 bits| 
| ``complex_``  | Abreviatura de ``complex128``.| 
| ``complex64`` | Número complejo, representado por dos floats de 32 bits| 
| ``complex128``| Número complejo, representado por dos floats de 64 bits| 

*un flotante de doble precisión permite representar números con mayor precisión y en un rango más amplio que un flotante de simple precisión debido a la mayor cantidad de bits utilizados para el exponente y la mantisa.*

<!--NAVIGATION-->
< [Introduccion a NumPy](0-Introduccion-a-NumPy.ipynb) | [Fundamentos de NumPy Arrays](2-Fundamentos_de_NumPy_Arrays.ipynb) >